In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

## Индекс демократии

In [ ]:
url = 'https://en.wikipedia.org/wiki/Democracy_Index'

source = requests.get(url)
main_text = source.text
soup = bs(main_text, 'html.parser')

In [ ]:
tables = [table.tbody for table in soup.find_all('table')]

In [ ]:
print(len(tables))
table = tables[6]

8


In [ ]:
dict_result = {'rank': [], 
               'country': [], 
               'regime_type': [], 
               'overall_score': [], 
               'electorat_and_pluralism': [], 
               'government': [], 
               'political_participation': [],
               'political_culture': [],
               'civil_liberties': []
               }

for i in table.find_all('tr')[3:]:
    try:
        dict_result['rank'].append(i.find_all('td')[0].text.replace("\n", ""))
        dict_result['country'].append(i.find_all('td')[2].text.replace("\n", "").replace("\xa0", ""))
        dict_result['regime_type'].append(i.find_all('td')[3].text.replace("\n", ""))
        dict_result['overall_score'].append(i.find_all('td')[4].text.replace("\n", ""))
        dict_result['electorat_and_pluralism'].append(i.find_all('td')[6].text.replace("\n", ""))
        dict_result['government'].append(i.find_all('td')[7].text.replace("\n", ""))
        dict_result['poli­tical_partici­pation'].append(i.find_all('td')[8].text.replace("\n", ""))
        dict_result['poli­tical_cul­ture'].append(i.find_all('td')[9].text.replace("\n", ""))
        dict_result['civil_liber­ties'].append(i.find_all('td')[10].text.replace("\n", ""))
    except:
        continue

In [ ]:
for key, value in dict_result.items():
    print(f'{value[0]} is {type(value[0])} type')

1 is <class 'str'> type
Norway is <class 'str'> type
Full democracy is <class 'str'> type
9.75 is <class 'str'> type
10.00 is <class 'str'> type
9.64 is <class 'str'> type
10.00 is <class 'str'> type
10.00 is <class 'str'> type
9.12 is <class 'str'> type


In [ ]:
democracy_index = pd.DataFrame(dict_result)
democracy_index.to_csv('parsing_tables/democracy_index.csv', index=False)

In [ ]:
colors = ['#0c3091', '#2f5cd5', '#6bd2df', '#c3eded', '#f9f8bb', '#fad45d', '#da820f', '#66000f']
colors_democracy_index_table = pd.DataFrame({'colors': colors})
colors_democracy_index_table.to_csv('parsing_tables/colors_democracy_index_table.csv', index=False)

## Индекс человеческого развития

In [3]:
url1 = 'https://en.wikipedia.org/wiki/List_of_countries_by_Human_Development_Index'

source1 = requests.get(url1)
main_text1 = source1.text
soup = bs(main_text1, 'html.parser')

In [4]:
tables = soup.find_all('table')
table = tables[1].tbody

In [64]:
hdi_index_dict = {'rank': [], 'country': [], 'hdi': [], 'average_annual_growth': []}

for row in table.find_all('tr')[2:]:
    try:
        if int(row.find_all('td')[0].text.replace("\n", '')):
            hdi_index_dict['rank'].append(row.find_all('td')[0].text.replace("\n", ''))
    except:
        hdi_index_dict['rank'].append(int(hdi_index_dict['rank'][-1])+1) 
    try:
        hdi_index_dict['country'].append(row.find('th').text.replace("\xa0", '').replace("\n", ''))
    except:
        continue
    try:
        hdi_index_dict['hdi'].append(row.find_all('td')[2].text)
    except:
        hdi_index_dict['hdi'].append(hdi_index_dict['hdi'][-1])
    if '%' in row.find_all('span')[2].text.replace(' ', ''):
        hdi_index_dict['average_annual_growth'].append(row.find_all('span')[2].text.replace(' ', ''))
    else:
        hdi_index_dict['average_annual_growth'].append(row.find_all('td')[3].text.replace(" ", '').replace("\n", ''))

In [66]:
df = pd.DataFrame(hdi_index_dict)

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191 entries, 0 to 190
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   rank                   191 non-null    object
 1   country                191 non-null    object
 2   hdi                    191 non-null    object
 3   average_annual_growth  191 non-null    object
dtypes: object(4)
memory usage: 6.1+ KB


In [72]:
df.nunique()

rank                     191
country                  191
hdi                      162
average_annual_growth     96
dtype: int64

In [73]:
df.to_csv('parsing_tables/human_dev_index.csv', index=False)

## Рейтинг качества жизни в городах

In [2]:
url = 'https://mobilityexchange.mercer.com/Insights/quality-of-living-rankings'

source = requests.get(url)
main_text = source.text
soup = bs(main_text, 'lxml')

In [11]:
all_cities = soup.tbody.contents[2:] + soup.find('div', id='fulltable').tbody.contents[2:]
dict_countries = {}
count_rank = 0
for i in all_cities:
    if i != '\n':
        ex_dict = {}
        ex_list = []
        for j in i.children:
            if j != '\n':
                ex_list.append(j.text)
        if ex_list[0] not in dict_countries:
            count_rank = 0
            ex_dict['City'] = ex_list[1]
            ex_dict['Country'] = ex_list[2]
            dict_countries[ex_list[0]] = ex_dict
        else:
            count_rank += 1
            ex_dict['City'] = ex_list[1]
            ex_dict['Country'] = ex_list[2]
            dict_countries[str(int(ex_list[0]) + count_rank)] = ex_dict
print(dict_countries['1'])
print(dict_countries['2'])

{'City': 'Vienna', 'Country': 'Austria'}
{'City': 'Zürich', 'Country': 'Switzerland'}


In [12]:
ranks = []
cities = []
countries = []
for key, value in dict_countries.items():
    ranks.append(int(key))
    cities.append(value['City'])
    countries.append(value['Country'])

In [13]:
quality_of_living_city_ranking = pd.DataFrame(data={'rank': ranks, 'city': cities, 'country': countries})

In [ ]:
quality_of_living_city_ranking.to_csv('parsing_tables/quality_of_living_city_index.csv', index=False)